In [1]:
import pandas as pd
from math import ceil
from datetime import timedelta

## Load Data from .csv

### Forecast by day with & shifts

In [4]:

shifts = pd.read_csv('shifts.csv', parse_dates=['Inicio_Turno','Fin_Turno'], date_parser=lambda x: pd.to_datetime(x, format="%H:%M"))
periodos = 24
tiempo = [(timedelta(hours=0, minutes=15*i * 96/periodos)) for i in range(int(periodos))]
shifts['in'] = shifts['Inicio_Turno'].apply(lambda x: tiempo.index(timedelta(hours=x.hour, minutes=x.minute)))
shifts['out'] = shifts['Fin_Turno'].apply(lambda x: len(tiempo) if x.hour==0 and x.minute==0 else tiempo.index(timedelta(hours=x.hour, minutes=x.minute)))
shifts['Lenght'] = shifts['out'] -shifts['in']

forecast_by_day = pd.read_csv('forecast_by_day.csv', parse_dates=['date'])
forecast_by_day['date'] = forecast_by_day['date'].astype('datetime64[ns]')
dates = forecast_by_day['date'].dt.date.unique()
forecast_by_day['week'] = forecast_by_day['date'].dt.isocalendar().week
forecast_by_day['day'] = forecast_by_day['date'].dt.isocalendar().day

/var/folders/k4/9s2d7pcj1j15dhx3p_kwf5mm0000gn/T/ipykernel_66548/2187563886.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  shifts = pd.read_csv('shifts.csv', parse_dates=['Inicio_Turno','Fin_Turno'], date_parser=lambda x: pd.to_datetime(x, format="%H:%M"))


## Parámetros
__k1 & k2__: (k1 findes libres cada k2 findes)  
__W_real__: cantidad de personas disponibles

In [5]:
k1= 1
k2 = 2
W_real = 9

max_wf_wk = forecast_by_day.query('day==7 or day==6')['Forecast'].max()
b1 = ceil((k2 * max_wf_wk) / (k2-k1))
D_week=0
for w in forecast_by_day['week'].unique():
    d = forecast_by_day[forecast_by_day['week']==w]['Forecast'].to_list()
    D=0
    for i in d:
        D += i
    if D > D_week:
        D_week = D
b2 = ceil(D/5)
b3 = forecast_by_day['Forecast'].max()

W = max(b1,b2,b3)
print(f"Minimum workforce: {W}")
if W_real > W:
    print("Workforce is enough, surplus: ", W_real-W)
    free_wf_wk = ceil(W_real*k1/k2)
else:
    print("Workforce is not enough, deficit: ", W-W_real)
    free_wf_wk = ceil(W_real*k1/k2)

print(f"Workforce free by Weekend: {free_wf_wk}")

Minimum workforce: 14
Workforce is not enough, deficit:  5
Workforce free by Weekend: 5


## Obtener superavit diario y agrupar colaborador por Domingo libre

In [6]:
max_demanda_per_weekend = {w:forecast_by_day[forecast_by_day['week']==w].query('day==7 or day==6')['Forecast'].max() for w in forecast_by_day['week'].unique()}
forecast_by_day['surplus'] = forecast_by_day.apply(lambda row: W_real - row['Forecast'] if row['day'] not in [6, 7] else W_real - max_demanda_per_weekend[row['week']], axis=1)
max_dda_wk = forecast_by_day[(forecast_by_day['day']==7) | (forecast_by_day['day']==6)]['Forecast'].max()

# Caso 1:2
collab_list = [i for i in range(W_real)]
collab_list_wknd = [collab_list[:int(len(collab_list)/2)], collab_list[int(len(collab_list)/2):]]
collab_list_wknd = collab_list_wknd*2

free_wknd_collab = {}
for w, cs in zip(max_demanda_per_weekend, collab_list_wknd):
    free_wknd_collab[w] = cs


## Generar Disponibilidad

In [7]:
def two_highest_keyvalue(dicc):
    dicc_copy = dict(dicc)
    key1 = max(dicc_copy, key=dicc_copy.get)
    del dicc_copy[key1]
    key2 = max(dicc_copy, key=dicc_copy.get)
    return [key1, key2]

In [8]:
data = []
for c in collab_list:
    for fecha in dates:
        data.append([c, fecha])

df_availability = pd.DataFrame(data, columns=['collaborator', 'date'])
df_availability['availability'] = 1
df_availability['date'] = df_availability['date'].astype('datetime64[ns]')
df_availability['week'] = df_availability['date'].dt.isocalendar().week
df_availability['day'] = df_availability['date'].dt.isocalendar().day

for w in free_wknd_collab: # Dar domingo libres
    for c in free_wknd_collab[w]:
        filtro = (df_availability["collaborator"] == c) & (df_availability["day"] == 7) & (df_availability["week"] == w)
        df_availability.loc[filtro, "availability"] = 0

day_off_by_week = {}
for wk in  free_wknd_collab: # Pares días libres
    surplus_week = forecast_by_day[(forecast_by_day['week']==wk) & (forecast_by_day['day']!=7)][['day','surplus']].set_index('day').to_dict()['surplus']
    day_off_pairs = []
    for j in range(len(collab_list) - len(free_wknd_collab[wk])):
        keys = two_highest_keyvalue(surplus_week)
        surplus_week[keys[0]] -= 1
        surplus_week[keys[1]] -= 1
        day_off_pairs.append(keys)

    day_off_by_week[wk] = day_off_pairs

for wk in day_off_by_week: # Dar libres entre semana
    j = 0
    for c in collab_list:
        if df_availability[(df_availability['collaborator']==c) & (df_availability['week']==wk)]['availability'].sum() == 7:
            days_off = day_off_by_week[wk][j]
            filter1 = (df_availability['collaborator']==c) & (df_availability['week']==wk) & (df_availability['day']==days_off[0])
            filter2 = (df_availability['collaborator']==c) & (df_availability['week']==wk) & (df_availability['day']==days_off[1])
            df_availability.loc[filter1, "availability"] = 0
            df_availability.loc[filter2, "availability"] = 0
            j+=1


In [9]:
df_availability

,collaborator,date,availability,week,day
0,0,2023-09-04,1,36,1
1,0,2023-09-05,1,36,2
2,0,2023-09-06,1,36,3
3,0,2023-09-07,1,36,4
4,0,2023-09-08,1,36,5
...,...,...,...,...,...
247,8,2023-09-27,1,39,3
248,8,2023-09-28,1,39,4
249,8,2023-09-29,1,39,5
250,8,2023-09-30,1,39,6
